In [22]:
import argparse
from pathlib import Path
from fastai.imagito.utils import *
from fastai.vision.models.xresnet2 import xresnet50_2
import torch
import math
from operator import itemgetter

from sklearn.model_selection import ParameterGrid
from fastai.imagito.utils import update_batch_size

to_grid = lambda p: update_batch_size(ParameterGrid(p))

In [13]:
def load_model(model_dir, size, woof, bs, sample):
#     model_params = pickle_load(model_dir + '/params.pkl')
#     if model_params['size'] != size:
#         print('Warning: Model was trained on image size=%d but is being used to predict for size=%d'
#               % (model_params['size'], size))

    m = xresnet50_2
    #print(model_params)
    data = get_data(size, woof, bs, sample, shuffle_train=False, flip_lr_p=0.0)
    l = Learner(data, m(c_out=10), path=model_dir)
    l.load('woof_classif')
    return l, data

In [ ]:
!mv woof_classif.pth

In [4]:
mkdir models

In [5]:
mv woof_classif.pth models

In [15]:
ls models

woof_classif.pth


In [19]:
model, data = load_model('.', 128, True, 256, 1.)

In [20]:
predictions_t, targets_t, loss_t = model.get_preds(ds_type=DatasetType.Train, with_loss=True)

In [21]:
to_arr = lambda x: x.numpy()

In [27]:
paths = list(map(lambda x: x.name,data.items))

In [33]:
df = pd.DataFrame(to_arr(loss_t), columns=['loss'])
df.index = paths

In [43]:
df = df.rename_axis('paths').sort_values('loss', ascending=False).reset_index()

In [47]:
df.head()

,paths,loss
0,n02086240_7862.JPEG,11.440737
1,n02089973_2860.JPEG,10.591484
2,n02088364_12006.JPEG,8.879482
3,n02086240_7549.JPEG,8.092752
4,n02093754_5276.JPEG,7.432945


In [19]:
from sklearn.model_selection import ParameterGrid
from fastai.imagito.utils import update_batch_size
from fastai.imagito.classes import ClassUtils

to_grid = lambda p: update_batch_size(ParameterGrid(p))

In [21]:
!git pull

Already up-to-date.


In [20]:
ClassUtils.from_indices()

['n02089973',
 'n02086240',
 'n02105641',
 'n02087394',
 'n02099601',
 'n02115641',
 'n02093754',
 'n02111889',
 'n02096294',
 'n02088364']

In [18]:
to_grid({'stem1': [16, 32, 48],
'stem2': [16, 32, 48],
'opt': ['SGD', 'RMS']})

[{'opt': 'SGD', 'stem1': 16, 'stem2': 16},
 {'opt': 'SGD', 'stem1': 16, 'stem2': 32},
 {'opt': 'SGD', 'stem1': 16, 'stem2': 48},
 {'opt': 'SGD', 'stem1': 32, 'stem2': 16},
 {'opt': 'SGD', 'stem1': 32, 'stem2': 32},
 {'opt': 'SGD', 'stem1': 32, 'stem2': 48},
 {'opt': 'SGD', 'stem1': 48, 'stem2': 16},
 {'opt': 'SGD', 'stem1': 48, 'stem2': 32},
 {'opt': 'SGD', 'stem1': 48, 'stem2': 48},
 {'opt': 'RMS', 'stem1': 16, 'stem2': 16},
 {'opt': 'RMS', 'stem1': 16, 'stem2': 32},
 {'opt': 'RMS', 'stem1': 16, 'stem2': 48},
 {'opt': 'RMS', 'stem1': 32, 'stem2': 16},
 {'opt': 'RMS', 'stem1': 32, 'stem2': 32},
 {'opt': 'RMS', 'stem1': 32, 'stem2': 48},
 {'opt': 'RMS', 'stem1': 48, 'stem2': 16},
 {'opt': 'RMS', 'stem1': 48, 'stem2': 32},
 {'opt': 'RMS', 'stem1': 48, 'stem2': 48}]

In [55]:
STEM_SETTINGS = [
    (SMALL, SMALL), (BIG, BIG),
    (SMALL, 32), (32, SMALL),
    (SMALL, BIG), (BIG, SMALL)
    #(4, 16), (16,4)
]
    
[dict(stem1=a, stem2=b) for (a,b) in STEM_SETTINGS]

[{'stem1': 16, 'stem2': 16},
 {'stem1': 48, 'stem2': 48},
 {'stem1': 16, 'stem2': 32},
 {'stem1': 32, 'stem2': 16},
 {'stem1': 16, 'stem2': 48},
 {'stem1': 48, 'stem2': 16}]

In [50]:
stem_grid = [{'stem1': x} for x in stem_testers]  + [{'stem2': x} for x in stem_testers] + [{'stem1': 16, 'stem2': 16}]
opt_grid = [{'opt': 'SGD', 'opt': 'RMS'}]

In [4]:
%load_ext autoreload
%autoreload 2

In [23]:
from fastai.imagito.grid_const import *
NEED_TO_RUN_ERIC_BOX_V3[1]

{'hardness_lower_bound': 0.0,
 'hardness_upper_bound': 0.25,
 'label_smoothing': False,
 'lr': 0.003,
 'flip_lr_p': 0.5,
 'woof': True}

In [6]:
len(NEED_TO_RUN_ERIC_BOX_V4)

48

In [3]:
!git pull

Already up-to-date.


In [51]:
stem_grid

[{'stem1': 16},
 {'stem1': 48},
 {'stem2': 16},
 {'stem2': 48},
 {'stem1': 16, 'stem2': 16}]

In [44]:
df.to_msgpack('/home/paperspace/fastai-fork/fastai/imagito/pred_df_woof.mp')

In [24]:
from fastai.imagito.train_imagito import main

In [ ]:
#main(woof=True, classes=range(4))

In [38]:
%debug

> /home/paperspace/anaconda3/envs/fastaiv1/lib/python3.6/site-packages/pandas/io/packers.py(226)read_msgpack()
    224         return read(path_or_buf)
    225 
--> 226     raise ValueError('path_or_buf needs to be a string file path or file-like')
    227 
    228 

ipdb> u
> /home/paperspace/fastai-fork/fastai/imagito/sample_hardness.py(139)make_hardness_filter_func()
    137     if (hardness_lower_bound == 0) and (hardness_upper_bound==1.):
    138         return return_true
--> 139     pred_df = pd.read_msgpack(path).sort_values('loss', ascending=False)
    140     start_idx, end_idx = int(IMAGENETTE_SIZE * hardness_lower_bound), int(
    141         IMAGENETTE_SIZE * hardness_upper_bound) - 1

ipdb> path
PosixPath('/home/paperspace/fastai-fork/fastai/imagito/pred_df_woof.mp')
ipdb> exit


In [32]:
df

,0
n02115641_44296.JPEG,0.112671
n02115641_7648.JPEG,0.235310
n02088364_1449.JPEG,0.863584
n02099601_8874.JPEG,0.386320
n02089973_4875.JPEG,1.649868
n02099601_447.JPEG,0.195452
n02115641_28502.JPEG,2.505424
n02093754_4460.JPEG,0.003552
n02093754_6370.JPEG,0.300552
n02086240_3672.JPEG,0.000826
